In [ ]:
library(GEOquery)

In [3]:
my_id <- "GSE178913"
gse_la <- getGEO(my_id)

Found 1 file(s)

GSE178913_series_matrix.txt.gz



In [10]:
gse <- gse_la[[1]]

In [13]:
gse <- read.table(
    'data/GSE211669/GSE211669_TMM.txt',
    sep = '\t', header = T,
    comment.char = '!'
)

In [14]:
View(gse)

GeneID,GSM6411618,GSM6411620,GSM6411621,GSM6411622,GSM6411624,GSM6411625,GSM6411626,GSM6411627,GSM6411628,⋯,MMAY00981_4_0,MWMH00083_4_1,MWMH00351_4_1,MWMH00475_4_3,MWMH00507_4_3,MWMH00552_4_9,MWMH00758_4_4,MWMH00758_7_8,MWMH00758_9_7,MWMH00851_4_0
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
TSPAN6,5.6895629,5.1623129,6.8922865,7.5380846,7.3373006,6.1270151,5.810375,7.15250453,5.755947,⋯,7.2624010,6.0354709,6.1734092,5.8544730,5.7205421,5.8219396,7.00563282,6.6733430,7.395836,7.4364552
TNMD,-0.3977654,-3.1165186,-3.1510872,-0.2954603,-2.1574685,-3.4094930,-1.715077,0.77821047,-3.241909,⋯,-1.4097909,1.0112153,-4.4246213,-4.4922525,-4.3419480,-2.9403929,-3.22358755,-4.2786633,-1.626862,-3.5109379
DPM1,6.2055118,5.2728007,5.9115760,5.8983419,6.6962502,5.2140718,5.441776,5.04251089,6.270614,⋯,5.6525720,5.0903934,6.3279646,5.8969427,5.4961833,4.7138399,6.44336138,6.5533651,6.515885,5.5969225
SCYL3,4.5160430,4.5014049,4.9352864,4.1170685,4.3181346,3.1826583,3.918693,3.34195656,4.552684,⋯,3.8250451,3.6904914,4.4820735,3.9414931,3.7276696,4.0443766,4.92303132,4.6791691,4.608563,4.2999255
C1orf112,4.0693357,4.7108808,3.3663039,3.6232913,4.8772919,1.9462437,3.459682,1.65880834,3.771396,⋯,3.3248432,3.6000180,4.3320117,3.6639138,3.4355528,4.2715675,4.49411794,4.1841259,3.824773,4.3867084
FGR,3.3139693,3.6735242,2.6264164,2.8474437,1.1616594,2.4331157,3.939762,-0.06848494,1.646420,⋯,2.3078453,4.3307107,1.9030079,2.8094334,3.4389699,3.9012149,3.28165006,3.6515425,4.128508,1.3851403
CFH,6.8544338,7.7846477,4.6786450,4.5786939,5.5357749,7.0124262,8.082541,5.04916753,6.770274,⋯,6.6874145,7.4546340,7.8407459,5.9389130,7.0019782,6.6875666,5.14055005,5.9847140,5.361517,6.4261220
FUCA2,5.2149852,6.0476342,6.1165678,6.4992395,6.6862115,6.6837345,5.377861,6.26416926,5.775658,⋯,6.2769806,5.5001276,5.4110844,5.8965949,5.9978530,6.0119518,5.36909234,5.9295091,6.174630,6.5077532
GCLC,5.7300170,4.9358926,5.5513770,5.4644342,5.4522159,5.0038846,5.383163,3.33385375,4.976657,⋯,4.5980876,4.8903708,4.6142878,5.5283483,4.6610107,5.2238740,5.04784795,5.0850546,4.973069,4.2545753
